In [3]:
#import sys
#sys.path.append('../..')
from scalers import scaler_identity
import pandas as pd
import os

log_path = os.path.join("logs","scalers_06")
part_res_path = os.path.join(log_path, "partial_results")

In [4]:
for f in os.listdir(part_res_path):
    print(f, end=", ")
for i in range(10):
    for f in os.listdir(part_res_path):
        #print(f)
        partial_results = pd.read_pickle(os.path.join(part_res_path,f))
        f_res = partial_results["results"]["kfolds"]
        #for res in f_res:
        print(f_res[i]["rmse_unscaled"], end=", ")
    print()

dasc_GlobalMaxAbsScaler().pkl, dasc_GlobalMinMaxScaler().pkl, dasc_GlobalRobustScaler().pkl, dasc_GlobalStandardScaler().pkl, dasc_MaxAbsScaler().pkl, dasc_MinMaxScaler().pkl, dasc_RobustScaler().pkl, dasc_scaler_identity.pkl, dasc_StandardScaler().pkl, 4.4999924, 2.6640496, 1.3823587, 0.8937101, 7.322883, 2.1035473, 2.4571037, 741.0723, 1.7732216, 
6.7602353, 8.396039, 2.2024038, 2.0627444, 6.8354125, 2.896182, 1.681365, 689.5244, 1.1771886, 
5.2405887, 9.380862, 1.3228341, 1.9082696, 5.6046576, 2.5838728, 2.1641343, 974.69696, 1.6653384, 
6.0699887, 3.3474123, 1.7604774, 1.8830723, 6.5858874, 4.1027727, 1.2340094, 915.20667, 1.3902448, 
6.5574675, 3.1637297, 2.0383534, 1.3318852, 6.5334873, 3.9163065, 2.6538682, 718.91315, 1.8766227, 
6.3239856, 2.8341167, 1.8640803, 1.2108804, 7.1328244, 1.9987758, 1.524344, 690.6599, 3.196848, 
8.338198, 4.9960604, 2.1041477, 1.1245036, 5.4060974, 1.9459037, 1.3688649, 464.41498, 2.6356137, 
15.3368635, 5.3661976, 1.7484703, 1.3605425, 6.4585104, 3

In [5]:
df_raw = None

for f in os.listdir(part_res_path):
    if "MaxAbs" in f or "MinMax" in f:
        continue
    partial_results = pd.read_pickle(os.path.join(part_res_path,f))
    res_folds = partial_results["results"]["kfolds"]
    rmse_unscaled = [fold["rmse_unscaled"] for fold in res_folds]
    
    temp_df = pd.DataFrame(rmse_unscaled).T  # Transpose the DataFrame to make it a row
    temp_df.columns = [f"fold_{i}" for i in range(len(rmse_unscaled))]
    scaler_name = f.replace(".pkl", "").replace("Scaler()","").replace("dasc_","")#.replace("Global","")
    temp_df.insert(0, 'scaler', scaler_name)  # Insert the scaler name as the first column
    
    df_raw = pd.concat([df_raw, temp_df], ignore_index=True)
df_raw = df_raw.set_index('scaler')
df_raw

,fold_0,fold_1,fold_2,fold_3,fold_4,fold_5,fold_6,fold_7,fold_8,fold_9
scaler,,,,,,,,,,
GlobalRobust,1.382359,2.202404,1.322834,1.760477,2.038353,1.864080,2.104148,1.748470,1.612839,1.917128
GlobalStandard,0.893710,2.062744,1.908270,1.883072,1.331885,1.210880,1.124504,1.360543,1.562981,1.731929
Robust,2.457104,1.681365,2.164134,1.234009,2.653868,1.524344,1.368865,1.237625,1.546696,2.338568
scaler_identity,741.072327,689.524414,974.696960,915.206665,718.913147,690.659912,464.414978,373.869659,943.326538,744.704224
Standard,1.773222,1.177189,1.665338,1.390245,1.876623,3.196848,2.635614,1.146284,1.195324,2.744275


In [6]:
from scipy import stats
import statsmodels.stats.multitest
from critdd import Diagram
from itertools import combinations
import numpy as np
import pandas as pd

unscaled_samples = {}  # Initialize an empty dictionary

for scaler, row in df_raw.iterrows():
    unscaled_samples[scaler] = row.values.tolist() * 3

#rmse_lists


In [7]:
samples = list(unscaled_samples.values())
print(np.array(samples).shape)
friedman_result = stats.friedmanchisquare(*samples)
friedman_result

(5, 30)


FriedmanchisquareResult(statistic=64.08000000000004, pvalue=4.020173888637316e-13)

In [8]:
combs = combinations(samples, 2)

pvals = []
for comb in combs:
    pvals.append(stats.wilcoxon(comb[0], comb[1]).pvalue)

pvals = np.array(pvals)
pvals.sort()
print(pvals)

for i in range(len(pvals)):
    if pvals[i] > 0.05/(len(pvals)-i):
        print(f"{pvals[i]} is not st. significant")
        print(f"{0.05/(len(pvals)-i)} is the threshold")
        break
    else:
        print(f"{pvals[i]} is st. significant")

statsmodels.stats.multitest.multipletests(pvals, method="holm")

[1.86264515e-09 1.86264515e-09 1.86264515e-09 1.86264515e-09
 7.29601830e-04 5.22633474e-02 6.05652314e-02 6.55438425e-01
 7.00032922e-01 7.45654674e-01]
1.862645149230957e-09 is st. significant
1.862645149230957e-09 is st. significant
1.862645149230957e-09 is st. significant
1.862645149230957e-09 is st. significant
0.0007296018302440643 is st. significant
0.052263347432017326 is not st. significant
0.01 is the threshold


(array([ True,  True,  True,  True,  True, False, False, False, False,
        False]),
 array([1.86264515e-08, 1.86264515e-08, 1.86264515e-08, 1.86264515e-08,
        4.37761098e-03, 2.61316737e-01, 2.61316737e-01, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00]),
 0.005116196891823743,
 0.005)

In [12]:
diag_df = pd.DataFrame(unscaled_samples)
diag_np = diag_df.to_numpy()
print(diag_np.shape)
print(len(diag_df.columns))

(30, 5)
5


In [13]:
diag_df = pd.DataFrame(unscaled_samples)
diag_np = diag_df.to_numpy()
diagram = Diagram(
    diag_df.to_numpy(),
    treatment_names=diag_df.columns,
    maximize_outcome = True,
)

#print(diagram.average_ranks)
#diagram.get_groups(alpha=0.05, adjustment="holm")
diagram.to_file(
    "diagram.tex",
    alpha=0.05,
    adjustment="holm",
    #reverse_x=True,
    #axis_options = {"title", "critdd"}
)